### Realizar el flujo que se hizo en el primero archivo, pero ahora para el archivo genre.csv

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [0]:
genre_schemas = StructType([
    StructField("genreId",IntegerType(), False),
    StructField("genreName", StringType(), True)
])


In [0]:
genre_df = spark.read \
    .option("header", True) \
    .schema(genre_schemas) \
    .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

In [0]:
genre_selection_df = genre_df.select("genreId", "genreName")
display(genre_selection_df)

In [0]:
genre_renamed_df = genre_selection_df.withColumnsRenamed({
    "genreId": "genre_id",
    "genreName": "genre_name"
})

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = genre_renamed_df.withColumns({
    "ingestion_date": current_timestamp(),
    "enviroment": lit("production"),
    "file_date": lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.genres")

In [0]:
overwrite_partition(genre_final_df, "movie_silver", "genres", "file_date")

In [0]:
#genre_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/genres")
#genre_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.genres")
genre_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.genres")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/genres

In [0]:
%sql
select file_date, count(1)
from movie_silver.genres
group by file_date;